In [1]:
import pandas
import numpy

In [ ]:
df = pandas.read_parquet("log_export.parquet")


In [11]:
df.shape

(11997652, 11)

In [7]:
df.head()

,date,ipsrc,ipdst,proto,portsrc,portdst,regle,action,interface_in,Interface_out,divers
0,2024-03-20 01:29:24,94.102.61.47,159.84.146.99,TCP,52502.0,3178.0,999.0,DENY,eth0,None,6.0
1,2024-03-20 01:29:25,176.111.174.85,159.84.146.99,TCP,48739.0,2231.0,999.0,DENY,eth0,None,6.0
2,2024-03-20 01:29:27,66.249.65.106,159.84.146.99,TCP,50501.0,443.0,1.0,PERMIT,eth0,None,6.0
3,2024-03-20 01:29:34,89.248.163.75,159.84.146.99,TCP,43312.0,8845.0,999.0,DENY,eth0,None,6.0
4,2024-03-20 01:29:38,42.58.163.244,159.84.146.99,TCP,9746.0,23.0,7.0,DENY,eth0,None,6.0


In [21]:
gb = df.groupby(by="ipsrc")


In [22]:
tab = pandas.DataFrame(index=gb.indices.keys())
tab.head(5)

""
""
1.0.128.207
1.0.158.106
1.0.159.170
1.0.159.58


In [33]:
tab['nombre'] = gb.size()

In [34]:
#nombre distinct d'ip de destination
tab['cnbipdst'] = gb.nunique()['ipdst']
tab.head(5)

,nombre,cnbipdst,cnportdst,Permit,Deny,infeq1024permit,adminpermit
,8,1,0,0,0,0,0
1.0.128.207,2,1,1,0,2,0,0
1.0.158.106,1,1,1,0,1,0,0
1.0.159.170,1,1,1,0,1,0,0
1.0.159.58,1,1,1,1,0,0,0


In [35]:
#nombre distinct de ports de destination
tab['cnportdst'] = gb.nunique()['portdst']
tab.head(5)

,nombre,cnbipdst,cnportdst,Permit,Deny,infeq1024permit,adminpermit
,8,1,0,0,0,0,0
1.0.128.207,2,1,1,0,2,0,0
1.0.158.106,1,1,1,0,1,0,0
1.0.159.170,1,1,1,0,1,0,0
1.0.159.58,1,1,1,1,0,0,0


In [36]:
temp = gb.value_counts(['action']).reset_index()
print(temp)
res = pandas.crosstab(temp['ipsrc'],temp['action'],temp['count'],aggfunc=lambda x:x)
res.head(15)
res = res.fillna(0)
res.head(15)
tab['Permit'] = res['PERMIT'].astype(int)
tab.head(15)
tab['Deny'] = res['DENY'].astype(int)
tab.head(15)

                ipsrc  action  count
0                                  8
1         1.0.128.207    DENY      2
2         1.0.158.106    DENY      1
3         1.0.159.170    DENY      1
4          1.0.159.58  PERMIT      1
...               ...     ...    ...
303331  99.99.118.204  PERMIT      1
303332    99.99.13.19  PERMIT      7
303333  99.99.193.220  PERMIT      1
303334    99.99.47.36  PERMIT      6
303335    99.99.67.24  PERMIT      3

[303336 rows x 3 columns]


,nombre,cnbipdst,cnportdst,Permit,Deny,infeq1024permit,adminpermit
,8,1,0,0,0,0,0
1.0.128.207,2,1,1,0,2,0,0
1.0.158.106,1,1,1,0,1,0,0
1.0.159.170,1,1,1,0,1,0,0
1.0.159.58,1,1,1,1,0,0,0
1.0.167.193,1,1,1,0,1,0,0
1.0.169.38,1,1,1,0,1,0,0
1.0.170.182,1,1,1,0,1,0,0
1.0.203.138,4,1,2,4,0,0,0
1.0.218.2,1,1,1,0,1,0,0


In [42]:
#permit -- ports inf à 1024
tab['infeq1024permit'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='PERMIT')*(sdf.portdst <= 1024)))
tab.head(15)

C:\Users\adardelet\AppData\Local\Temp\ipykernel_29524\400339359.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tab['infeq1024permit'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='PERMIT')*(sdf.portdst <= 1024)))


,nombre,cnbipdst,cnportdst,Permit,Deny,infeq1024permit,adminpermit,sup1024permit,infeq1024deny,sup1024deny
,8,1,0,0,0,0,0,0,0,0
1.0.128.207,2,1,1,0,2,0,0,0,0,0
1.0.158.106,1,1,1,0,1,0,0,0,0,0
1.0.159.170,1,1,1,0,1,0,0,0,0,0
1.0.159.58,1,1,1,1,0,1,0,0,0,0
1.0.167.193,1,1,1,0,1,0,0,0,0,0
1.0.169.38,1,1,1,0,1,0,0,0,0,0
1.0.170.182,1,1,1,0,1,0,0,0,0,0
1.0.203.138,4,1,2,4,0,4,0,0,0,0
1.0.218.2,1,1,1,0,1,0,0,0,0,0


In [43]:
#admin. permit
tab['adminpermit'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='PERMIT')*(sdf.portdst.isin([21,22,3389,3306]))))
tab.head(20)

C:\Users\adardelet\AppData\Local\Temp\ipykernel_29524\590987997.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tab['adminpermit'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='PERMIT')*(sdf.portdst.isin([21,22,3389,3306]))))


,nombre,cnbipdst,cnportdst,Permit,Deny,infeq1024permit,adminpermit,sup1024permit,infeq1024deny,sup1024deny
,8,1,0,0,0,0,0,0,0,0
1.0.128.207,2,1,1,0,2,0,0,0,0,0
1.0.158.106,1,1,1,0,1,0,0,0,0,0
1.0.159.170,1,1,1,0,1,0,0,0,0,0
1.0.159.58,1,1,1,1,0,1,0,0,0,0
1.0.167.193,1,1,1,0,1,0,0,0,0,0
1.0.169.38,1,1,1,0,1,0,0,0,0,0
1.0.170.182,1,1,1,0,1,0,0,0,0,0
1.0.203.138,4,1,2,4,0,4,0,0,0,0
1.0.218.2,1,1,1,0,1,0,0,0,0,0


In [44]:
#permit -- ports sup à 1024
tab['sup1024permit'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='PERMIT')*(sdf.portdst > 1024)))
tab.head(10)

C:\Users\adardelet\AppData\Local\Temp\ipykernel_29524\2518835589.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tab['sup1024permit'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='PERMIT')*(sdf.portdst > 1024)))


,nombre,cnbipdst,cnportdst,Permit,Deny,infeq1024permit,adminpermit,sup1024permit,infeq1024deny,sup1024deny
,8,1,0,0,0,0,0,0,0,0
1.0.128.207,2,1,1,0,2,0,0,0,0,0
1.0.158.106,1,1,1,0,1,0,0,0,0,0
1.0.159.170,1,1,1,0,1,0,0,0,0,0
1.0.159.58,1,1,1,1,0,1,0,0,0,0
1.0.167.193,1,1,1,0,1,0,0,0,0,0
1.0.169.38,1,1,1,0,1,0,0,0,0,0
1.0.170.182,1,1,1,0,1,0,0,0,0,0
1.0.203.138,4,1,2,4,0,4,0,0,0,0
1.0.218.2,1,1,1,0,1,0,0,0,0,0


In [45]:
#deny -- ports inf à 1024
tab['infeq1024deny'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='DENY')*(sdf.portdst <= 1024)))
tab.head(15)

C:\Users\adardelet\AppData\Local\Temp\ipykernel_29524\3198734577.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tab['infeq1024deny'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='DENY')*(sdf.portdst <= 1024)))


,nombre,cnbipdst,cnportdst,Permit,Deny,infeq1024permit,adminpermit,sup1024permit,infeq1024deny,sup1024deny
,8,1,0,0,0,0,0,0,0,0
1.0.128.207,2,1,1,0,2,0,0,0,2,0
1.0.158.106,1,1,1,0,1,0,0,0,1,0
1.0.159.170,1,1,1,0,1,0,0,0,1,0
1.0.159.58,1,1,1,1,0,1,0,0,0,0
1.0.167.193,1,1,1,0,1,0,0,0,1,0
1.0.169.38,1,1,1,0,1,0,0,0,0,0
1.0.170.182,1,1,1,0,1,0,0,0,1,0
1.0.203.138,4,1,2,4,0,4,0,0,0,0
1.0.218.2,1,1,1,0,1,0,0,0,1,0


In [46]:
#deny -- ports sup à 1024
tab['sup1024deny'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='DENY')*(sdf.portdst > 1024)))
tab.head(10)

C:\Users\adardelet\AppData\Local\Temp\ipykernel_29524\338189329.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tab['sup1024deny'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='DENY')*(sdf.portdst > 1024)))


,nombre,cnbipdst,cnportdst,Permit,Deny,infeq1024permit,adminpermit,sup1024permit,infeq1024deny,sup1024deny
,8,1,0,0,0,0,0,0,0,0
1.0.128.207,2,1,1,0,2,0,0,0,2,0
1.0.158.106,1,1,1,0,1,0,0,0,1,0
1.0.159.170,1,1,1,0,1,0,0,0,1,0
1.0.159.58,1,1,1,1,0,1,0,0,0,0
1.0.167.193,1,1,1,0,1,0,0,0,1,0
1.0.169.38,1,1,1,0,1,0,0,0,0,1
1.0.170.182,1,1,1,0,1,0,0,0,1,0
1.0.203.138,4,1,2,4,0,4,0,0,0,0
1.0.218.2,1,1,1,0,1,0,0,0,1,0


In [47]:
#admin. permit
tab['admindeny'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='DENY')*(sdf.portdst.isin([21,22,3389,3306]))))
tab.head(20)

C:\Users\adardelet\AppData\Local\Temp\ipykernel_29524\848965699.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tab['admindeny'] = gb.apply(lambda sdf:numpy.sum((sdf.action=='DENY')*(sdf.portdst.isin([21,22,3389,3306]))))


,nombre,cnbipdst,cnportdst,Permit,Deny,infeq1024permit,adminpermit,sup1024permit,infeq1024deny,sup1024deny,admindeny
,8,1,0,0,0,0,0,0,0,0,0
1.0.128.207,2,1,1,0,2,0,0,0,2,0,0
1.0.158.106,1,1,1,0,1,0,0,0,1,0,0
1.0.159.170,1,1,1,0,1,0,0,0,1,0,0
1.0.159.58,1,1,1,1,0,1,0,0,0,0,0
1.0.167.193,1,1,1,0,1,0,0,0,1,0,0
1.0.169.38,1,1,1,0,1,0,0,0,0,1,0
1.0.170.182,1,1,1,0,1,0,0,0,1,0,0
1.0.203.138,4,1,2,4,0,4,0,0,0,0,0
1.0.218.2,1,1,1,0,1,0,0,0,1,0,0
